# SentencePiece
- https://wikidocs.net/86657
- 구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 깃허브에 공개

## Import

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.1 MB 5.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/user/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

## Load dataset

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f8f66ac60f0>)

In [7]:
train_df = pd.read_csv('IMDb_Reviews.csv')
print(f"Num of rows: {len(train_df)}")
train_df.head()

Num of rows: 50000


,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


## Test SentencePiece
input : 학습시킬 파일
model_prefix : 만들어질 모델 이름
vocab_size : 단어 집합의 크기
model_type : 사용할 모델 (unigram(default), bpe, char, word)
max_sentence_length: 문장의 최대 길이
pad_id, pad_piece: pad token id, 값
unk_id, unk_piece: unknown token id, 값
bos_id, bos_piece: begin of sentence token id, 값
eos_id, eos_piece: end of sequence token id, 값
user_defined_symbols: 사용자 정의 토큰

In [8]:
# 센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [9]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [13]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
print(f"Num of words: {len(vocab_list)}")
vocab_list.sample(10)

Num of words: 5000


,0,1
446,ually,-443
1799,▁chance,-1796
1943,ina,-1940
4422,▁issue,-4419
141,▁have,-138
3410,▁neigh,-3407
3056,▁amusing,-3053
4123,▁provide,-4120
4345,▁Pal,-4342
1045,▁actor,-1042


In [11]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

- encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환합니다.
- encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환합니다.
- GetPieceSize() : 단어 집합의 크기를 확인합니다.
- idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.
- PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.
- DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.
- DecodePieces : 서브워드 시퀀스로부터 문장으로 변환합니다.
- encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.

In [14]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [15]:
# Num of words
sp.GetPieceSize()

5000

In [16]:
# Change index to word
sp.IdToPiece(430)

'▁character'

In [17]:
# Change word to index
sp.PieceToId('▁character')

430

In [18]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [19]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [20]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


# SentencePiece를 사용해서 네이버 영화 리뷰 토큰화하기